# ROUTINE CLEANING (HEBDOMADAIRE)

In [90]:
import pandas as pd

In [91]:
#Lecture de la BDD intermédiaire rassemblant les tconsts des films retenus pour nos BDDs
df_tconst = pd.read_csv('tconst.csv', sep=',')
df_tconst.rename(columns = {'titleId':'tconst'}, inplace=True)

# TITLE BASICS

In [92]:
title_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz'
dfTB = pd.read_csv(title_basics, sep='\t')

dfTB['isAdult'] = pd.to_numeric(dfTB['isAdult'], errors='coerce')
dfTB['endYear'] = pd.to_numeric(dfTB['endYear'], errors='coerce')
dfTB['startYear'] = pd.to_numeric(dfTB['startYear'], errors='coerce')

## Replacer les années start/endYear qui ont été rentré dans la colonne isadult

dfTBA = dfTB[dfTB['isAdult']>1]
dfTBA2 = dfTBA.loc[:,['tconst', 'isAdult']]
for i in range(len(dfTBA2)):
    x = dfTBA2.iloc[i,1]
    dfTB['startYear'][dfTB['tconst'] == dfTBA2.iloc[i,0]] = x
    dfTB['isAdult'][dfTB['tconst'] == dfTBA2.iloc[i,0]] = 0
del dfTBA
del dfTBA2 

## Replacer les genres qui ont été rentré dans la colonne run Times
dfTBA = dfTB[dfTB['runtimeMinutes'].str.contains("[a-z]") | dfTB['runtimeMinutes'].str.contains("[A-I]") ]
dfTBA2 = dfTBA.loc[:,['tconst', 'runtimeMinutes']]
for i in range(len(dfTBA2)):
    x = dfTBA2.iloc[i,1]
    dfTB['genres'][dfTB['tconst'] == dfTBA2.iloc[i,0]] = x
    dfTB['runtimeMinutes'][dfTB['tconst'] == dfTBA2.iloc[i,0]] = 0
del dfTBA
del dfTBA2
## Optimisation des types des colonnes numériques
dfTB['runtimeMinutes'] = pd.to_numeric(dfTB['runtimeMinutes'], errors='coerce')

dfTB['startYear'].fillna(0, inplace = True)
dfTB['isAdult'].fillna(0, inplace = True)  # à voir si on considère les Nan comme films pour adultes ou non 
dfTB['endYear'].fillna(0, inplace = True)
dfTB['runtimeMinutes'].fillna(0, inplace = True)

dfTB['startYear']=dfTB['startYear'].astype(int, errors='ignore')
dfTB['isAdult']=dfTB['isAdult'].astype(bool, errors='ignore')
dfTB['endYear']=dfTB['endYear'].astype(int, errors='ignore')
dfTB['runtimeMinutes'] = dfTB['runtimeMinutes'].astype(int, errors='ignore')

dfTB['endYear'] = pd.to_numeric(dfTB['endYear'],downcast='integer',errors='ignore')
dfTB['startYear'] = pd.to_numeric(dfTB['startYear'],downcast='integer',errors='ignore')
dfTB['runtimeMinutes'] = pd.to_numeric(dfTB['runtimeMinutes'],downcast='integer',errors='ignore')

dfTBOK=dfTB.loc[dfTB['isAdult']==False]

dfTB2 = pd.merge(df_tconst, dfTBOK, how='left', on='tconst')
dfTB2 = dfTB2.drop_duplicates()

dfTB2.to_csv('title.basics2.tsv', sep='\t', index=False)

/var/folders/vs/6ltrzv_x4w1d9xggkvn3wzd40000gn/T/ipykernel_60934/1450652348.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTB['startYear'][dfTB['tconst'] == dfTBA2.iloc[i,0]] = x
/var/folders/vs/6ltrzv_x4w1d9xggkvn3wzd40000gn/T/ipykernel_60934/1450652348.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTB['isAdult'][dfTB['tconst'] == dfTBA2.iloc[i,0]] = 0
/var/folders/vs/6ltrzv_x4w1d9xggkvn3wzd40000gn/T/ipykernel_60934/1450652348.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

#TITLE PRINCIPALS

In [93]:
title_principals = 'https://datasets.imdbws.com/title.principals.tsv.gz'
df_title_principals = pd.read_csv(title_principals, sep='\t')

df_title_principals['job'].loc[df_title_principals['job']==r'\N']= None
df_title_principals['characters'].loc[df_title_principals['characters']==r'\N']= None

dfTP2 = pd.merge(df_tconst, df_title_principals, how='left', on='tconst')
dfTP2 = dfTP2.drop_duplicates()

dfTP2.to_csv('title.principals2.tsv', sep='\t',index=False)

#Sur cette table, on remplace simplement les valeurs \N par des vrais 'None' pour éviter que l'ordinateur perde du temps à parcourir des lignes qui sont en réalité vides

#TITLE RATINGS

In [94]:
title_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
df_title_ratings = pd.read_csv(title_ratings, sep='\t')

#Ces colonnes doivent être converties en integer pour être correctement interprétées. 
df_title_ratings['numVotes'] = pd.to_numeric(df_title_ratings['numVotes'],downcast='integer',errors='ignore')
df_title_ratings['averageRating'] = pd.to_numeric(df_title_ratings['averageRating'],downcast='float',errors='ignore')

dfTR2 = pd.merge(df_tconst, df_title_ratings, how='left', on='tconst')
dfTR2 = dfTR2.drop_duplicates()

dfTR2.to_csv('title.ratings2.tsv', sep='\t', index=False)

#TITLE AKAS

In [95]:
title_akas = 'https://datasets.imdbws.com/title.akas.tsv.gz'
df_title_akas = pd.read_csv(title_akas, sep='\t')

df_title_akas['isOriginalTitle']=pd.to_numeric(df_title_akas['isOriginalTitle'], errors ='coerce')
df_title_akas.fillna(1, inplace = True)
df_title_akas['isOriginalTitle'] = df_title_akas['isOriginalTitle'].astype(bool, errors='ignore')

df_title_akas['language'].loc[df_title_akas['language']==r'\N']= None
df_title_akas['types'].loc[df_title_akas['types']==r'\N']= None
df_title_akas['attributes'].loc[df_title_akas['attributes']==r'\N']= None
 
df_french = df_title_akas.loc[df_title_akas['region']=='FR']
df_french.to_csv('title.akas2.tsv', sep='\t', index=False )
#df_title_akas.to_csv('title.akas2.tsv', sep='\t', index=False)

#NAME BASICS

In [96]:
name_basics = 'https://datasets.imdbws.com/name.basics.tsv.gz'
df_name_basics = pd.read_csv(name_basics, sep='\t')

df_name_basics['birthYear'] = pd.to_numeric(df_name_basics['birthYear'], errors='coerce')
df_name_basics['birthYear'].loc[df_name_basics['birthYear']==0]= None
df_name_basics['birthYear']=df_name_basics['birthYear'].astype(int, errors='ignore')
df_name_basics['birthYear'] = pd.to_numeric(df_name_basics['birthYear'],downcast='integer',errors='ignore')

df_name_basics['deathYear'] = pd.to_numeric(df_name_basics['deathYear'], errors='coerce')
df_name_basics['deathYear'].loc[df_name_basics['deathYear']==0]= None
df_name_basics['deathYear']=df_name_basics['deathYear'].astype(int, errors='ignore')
df_name_basics['deathYear'] = pd.to_numeric(df_name_basics['deathYear'],downcast='integer',errors='ignore')

df_name_basics.to_csv('name.basics2.tsv', sep='\t', index=False)


# TITLE CREW

In [97]:
title_crew = 'https://datasets.imdbws.com/title.crew.tsv.gz'
df_title_crew = pd.read_csv(title_crew, sep='\t')

df_title_crew['writers'].loc[df_title_crew['writers']==r'\N']= None
df_title_crew['directors'].loc[df_title_crew['directors']==r'\N']= None

df_title_crew.to_csv('title.crew2.tsv', sep='\t', index=False)

# TITLE EPISODE

In [98]:
title_episode = 'https://datasets.imdbws.com/title.episode.tsv.gz'
df_title_episode = pd.read_csv(title_episode, sep='\t')

df_title_episode['episodeNumber'].loc[df_title_episode['episodeNumber']==r'\N']= None
df_title_episode['seasonNumber'].loc[df_title_episode['seasonNumber']==r'\N']= None

df_title_episode.to_csv('title.episode2.tsv', sep='\t', index=False)